<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

>>>>>>>>>>>>> # Featurization

In [1]:
import pickle
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import networkx as nx
from tqdm import tqdm
from scipy.sparse.linalg import svds

# 1. Reading Data

In [24]:
X_train = pd.read_csv('datasets/my/xtrain.csv',header=None)
Y_train = pd.read_csv('datasets/my/ytrain.csv',header=None)
X_test = pd.read_csv('datasets/my/xtest.csv',header=None)
Y_test = pd.read_csv('datasets/my/ytest.csv',header=None)
Y_train.shape,X_train.shape,Y_test.shape,X_test.shape

((15100030, 1), (15100030, 2), (3775008, 1), (3775008, 2))

graph created using the positive training data as the negative data consists of unconnected edges which is useless while creating graph

In [3]:
tg=nx.read_edgelist('datasets/data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int) 

# 2. Similarity measures

## 2.1 Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

### 2.1.1 For Followee (i.e. the source and destination nodes are Followee)

In [4]:
def jacc_followee(u,v,g):
    try:
        u_followers  = set([ x for (x,y) in g.in_edges(u)])
        v_followers  = set([ x for (x,y) in g.in_edges(v)])
        if len(u_followers)==0 | len(v_followers)==0:
            return 0
        return len(u_followers.intersection(v_followers))/len(u_followers.union(v_followers))
    except:
        return 0 

### 2.1.2 For Follower (i.e. the source and destination nodes are Followers)

In [5]:
def jacc_follower(u,v,g):
    try:
        u_followees  = set([ y for (x,y) in g.out_edges(u)])
        v_followees  = set([ y for (x,y) in g.out_edges(v)])
        if len(u_followees)==0 | len(v_followees)==0:
            return 0
        return len(u_followees.intersection(v_followees))/len(u_followees.union(v_followees))
    except:
        return 0 

### 2.1.3 AAI Functions

In [6]:
# #for followees
# def jaccard_for_followees(a,b):
#     try:
#         if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
#             return 0
#         sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
#                                     (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
#     except:
#         return 0
#     return sim
    
# #for followers
# def jaccard_for_followers(a,b):
#     try:
#         if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
#             return 0
#         sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
#                                  (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
#         return sim
#     except:
#         return 0

## 2.2 Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}
it has an

In [7]:
# u,v being followee
def cosine_followee(u,v,g):
    try:
        u_followers  = set([ x for (x,y) in g.in_edges(u)])
        v_followers  = set([ x for (x,y) in g.in_edges(v)])
        if len(u_followers)==0 | len(v_followers)==0:
            return 0
        return len(u_followers.intersection(v_followers))/math.sqrt(len(u_followers)*len(v_followers))
    except:
        return 0 
# u,v being followers
def cosine_follower(u,v,g):
    try:
        u_followees  = set([ y for (x,y) in g.out_edges(u)])
        v_followees  = set([ y for (x,y) in g.out_edges(v)])
        if len(u_followees)==0 | len(v_followees)==0:
            return 0
        return len(u_followees.intersection(v_followees))/math.sqrt(len(u_followees)*len(v_followees))
    except:
        return 0 

### AAI functions

In [8]:
# #for followees
# def cosine_for_followees(a,b):
#     try:
#         if len(set(tg.successors(a))) == 0  | len(set(tg.successors(b))) == 0:
#             return 0
#         sim = (len(set(tg.successors(a)).intersection(set(tg.successors(b)))))/\
#                                     (math.sqrt(len(set(tg.successors(a)))*len((set(tg.successors(b))))))    
#         return sim
#     except:
#         return 0

# # for followers 
# def cosine_for_followers(a,b):
#     try:
        
#         if len(set(tg.predecessors(a))) == 0  | len(set(tg.predecessors(b))) == 0:
#             return 0
#         sim = (len(set(tg.predecessors(a)).intersection(set(tg.predecessors(b)))))/\
#                                      (math.sqrt(len(set(tg.predecessors(a))))*(len(set(tg.predecessors(b)))))
#         return sim
#     except:
#         return 0


## 3. Ranking Measures

https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.

<img src='PageRanks-Example.jpg' height=300px />

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

https://en.wikipedia.org/wiki/PageRank

### ` page rank is fairly based on complex graph theory, so we are not going to go deep here about various aspects of pagerank like 'alhpa value' etc.` 

In [9]:
if not os.path.isfile('datasets/pickle/page_rank.p'):
    pr = nx.pagerank(tg, alpha=0.85)
    pickle.dump(pr,open('datasets/pickle/page_rank.p','wb'))
else:
    pr = pickle.load(open('datasets/pickle/page_rank.p','rb'))

In [10]:
type(pr)

dict

In [11]:
print('min',min(list(pr.values())))
print('max',max(list(pr.values())))
print('mean',float(sum(pr.values())) / len(pr))

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


In [12]:
#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.615699699389075e-07


 # 4. Other Graph Features

## 4.1 Shortest path:

Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [13]:
def shortest_path(u,v,g):
    try:
        if not nx.has_path(g,u,v):
            return -1
        elif g.has_edge(u,v):
            g.remove_edge(u,v)
            p = nx.shortest_path_length(g,source=u,target=v)
            g.add_edge(u,v)
            return p
        else:
            return nx.shortest_path_length(g,source=u,target=v)
    except:
        return -1

## 4.2 Checking for same community

In [14]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(tg))
def same_wcc(u,v):
    index = []
    if tg.has_edge(v,u):
        return 1
    if tg.has_edge(u,v):
            for i in wcc:
                if u in i:
                    index= i
                    break
            if (v in index):
                tg.remove_edge(u,v)
                if shortest_path(u,v,tg)==-1:
                    tg.add_edge(u,v)
                    return 0
                else:
                    tg.add_edge(u,v)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if u in i:
                    index= i
                    break
            if(v in index):
                return 1
            else:
                return 0

## 4.3 Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [15]:
#adar index
def adar_index(u,v):
    sum=0
    try:
        n=list(set(tg.successors(u)).intersection(set(tg.successors(v))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(tg.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

## 4.4 Is persion was following back:

In [16]:
def follow_back(u,v):
    try:
        if tg.has_edge(v,u):
            return 1
        else: 
            return 0 
    except:
        return 0 

## 4.5 Katz Centrality:
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [17]:
if not os.path.isfile('datasets/pickle/katz.p'):
    katz = nx.katz.katz_centrality(tg,alpha=0.005,beta=1)
    pickle.dump(katz,open('datasets/pickle/katz.p','wb'))
else:
    katz = pickle.load(open('datasets/pickle/katz.p','rb'))

print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
# print('mean',float(sum(katz.values())) / len(katz))

mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

min 0.0007313532484065916
max 0.003394554981699122
0.0007483800935562018


## 4.6 Hits Score
The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

https://en.wikipedia.org/wiki/HITS_algorithm

In [18]:
if not os.path.isfile('datasets/pickle/hits.p'):
    hits = nx.hits(tg, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('datasets/pickle/hits.p','wb'))
else:
    hits = pickle.load(open('datasets/pickle/hits.p','rb'))

print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 5. Featurization

## 5. 1 Reading a sample of Data from both train and test

In [25]:
X_train['y'] = Y_train.astype('int32')[0]

In [26]:
X_train.head(5)

,0,1,y
0,273084,1505602,1
1,912810,1678443,1
2,365429,1523458,1
3,527014,1605979,1
4,1228116,471233,1


In [27]:
X_test['y'] = Y_test.astype('int32')[0]

In [28]:
X_test.head(5)

,0,1,y
0,848424,784690,1
1,1248963,444518,1
2,264224,132395,1
3,549680,326829,1
4,875380,1394902,1


### Sampling form the dataset

In [29]:
xtrain = X_train.sample(n = 100000,replace = False,random_state = 2)
xtrain[xtrain['y']==0].shape,xtrain[xtrain['y']==1].shape

((50092, 3), (49908, 3))

In [30]:
xtest = X_test.sample(n = 50000,replace = False,random_state = 2)
xtest[xtest['y']==0].shape,xtest[xtest['y']==1].shape

((24971, 3), (25029, 3))

In [31]:
xtrain.head(5)

,0,1,y
1978364,488489,1053487,1
7556568,1248019,995936,0
2708232,1166276,431589,1
13693456,1707801,977540,0
10614818,168329,615541,0


## 5.2 Adding a set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>jaccard_followers</li>
<li>jaccard_followees</li>
<li>cosine_followers</li>
<li>cosine_followees</li>
<li>num_followers_s</li>
<li>num_followees_s</li>
<li>num_followers_d</li>
<li>num_followees_d</li>
<li>inter_followers</li>
<li>inter_followees</li>
</ol>

In [32]:

# Faeture 1,2,3,4

xtrain['jacc_followee']  = xtrain.apply(lambda x:jacc_followee(x[0],x[1],tg),axis=1)
xtrain['jacc_follower']  = xtrain.apply(lambda x:jacc_follower(x[0],x[1],tg),axis=1)
xtrain['cosine_followee']  = xtrain.apply(lambda x:cosine_followee(x[0],x[1],tg),axis=1)
xtrain['cosine_follower']  = xtrain.apply(lambda x:cosine_follower(x[0],x[1],tg),axis=1)

xtest['jacc_followee']  = xtest.apply(lambda x:jacc_followee(x[0],x[1],tg),axis=1)
xtest['jacc_follower']  = xtest.apply(lambda x:jacc_follower(x[0],x[1],tg),axis=1)
xtest['cosine_followee']  = xtest.apply(lambda x:cosine_followee(x[0],x[1],tg),axis=1)
xtest['cosine_follower']  = xtest.apply(lambda x:cosine_follower(x[0],x[1],tg),axis=1)

In [33]:
# feature 5 i.e no. of followers of the u
def foller(n):
    try:
        return len(set(tg.predecessors(n)))
    except:
        return 0 
xtrain['num_followers_s'] = xtrain.apply(lambda x:foller(x[0]),axis=1)
xtest['num_followers_s'] = xtest.apply(lambda x:foller(x[0]),axis=1)

# feature 7 i.e no. of followers of the v
xtrain['num_followers_d'] = xtrain.apply(lambda x:foller(x[1]),axis=1)
xtest['num_followers_d'] = xtest.apply(lambda x:foller(x[1]),axis=1)

In [34]:
# feature 6 and 8 i.e no of nodes u,v follow
def folloee(n):
    try:
        return len(set(tg.successors(n)))
    except:
        return 0 
xtrain['num_followees_s'] = xtrain.apply(lambda x:folloee(x[0]),axis=1)
xtest['num_followees_s'] = xtest.apply(lambda x:folloee(x[0]),axis=1)

xtrain['num_followees_d'] = xtrain.apply(lambda x:folloee(x[1]),axis=1)
xtest['num_followees_d'] = xtest.apply(lambda x:folloee(x[1]),axis=1)

In [35]:
# feature 9
def intre_follower(u,v):
    try:
        return len(set(tg.predecessors(u)).intersection(set(tg.predecessors(v))))
    except:
        return 0
xtrain['inter_followers'] = xtrain.apply(lambda x:intre_follower(x[0],x[1]),axis=1)
xtest['inter_followers'] = xtest.apply(lambda x:intre_follower(x[0],x[1]),axis=1)

In [36]:
# feature 10
def intre_followee(u,v):
    try:
        return len(set(tg.successors(u)).intersection(set(tg.successors(v))))
    except:
        return 0
xtrain['inter_followees'] = xtrain.apply(lambda x:intre_followee(x[0],x[1]),axis=1)
xtest['inter_followees'] = xtest.apply(lambda x:intre_followee(x[0],x[1]),axis=1)

## 5.3 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>adar index</li>
<li>is following back</li>
<li>belongs to same weakly connect components</li>
<li>shortest path between source and destination</li>
</ol>

In [37]:
# feature 1
xtrain['adar_index'] = xtrain.apply(lambda x:adar_index(x[0],x[1]),axis=1)
xtest['adar_index'] = xtest.apply(lambda x:adar_index(x[0],x[1]),axis=1)

In [38]:
# feature 2
xtrain['follow_back'] = xtrain.apply(lambda x:follow_back(x[0],x[1]),axis=1)
xtest['follow_back'] = xtest.apply(lambda x:follow_back(x[0],x[1]),axis=1)

In [39]:
# feature 3
xtrain['same_wcc'] = xtrain.apply(lambda x:same_wcc(x[0],x[1]),axis=1)
xtest['same_wcc'] = xtest.apply(lambda x:same_wcc(x[0],x[1]),axis=1)

In [51]:
# feature 4
xtrain['shortest_path'] = xtrain.apply(lambda x:shortest_path(x[0],x[1],tg),axis=1)
xtest['shortest_path'] = xtest.apply(lambda x:shortest_path(x[0],x[1],tg),axis=1)

## 5.4 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>Weight Features
    <ul>
        <li>weight of incoming edges</li>
        <li>weight of outgoing edges</li>
        <li>weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges * weight of outgoing edges</li>
        <li>2*weight of incoming edges + weight of outgoing edges</li>
        <li>weight of incoming edges + 2*weight of outgoing edges</li>
    </ul>
</li>
<li>Page Ranking of source</li>
<li>Page Ranking of dest</li>
<li>katz of source</li>
<li>katz of dest</li>
<li>hubs of source</li>
<li>hubs of dest</li>
<li>authorities_s of source</li>
<li>authorities_s of dest</li>
</ol>

#### Weight Features
In order to determine the similarity of nodes, an edge weight value was calculated between nodes. Edge weight decreases as the neighbor count goes up. Intuitively, consider one million people following a celebrity on a social network then chances are most of them never met each other or the celebrity. On the other hand, if a user has 30 contacts in his/her social network, the chances are higher that many of them know each other. 
`credit` - Graph-based Features for Supervised Link Prediction
William Cukierski, Benjamin Hamner, Bo Yang

\begin{equation}
W = \frac{1}{\sqrt{1+|X|}}
\end{equation}

it is directed graph so calculated Weighted in and Weighted out differently

In [52]:
#weight for source and destination of each link
Weight_in = {}
Weight_out = {}
for i in  tqdm(tg.nodes()):
    s1=set(tg.predecessors(i))
    w_in = 1.0/(np.sqrt(1+len(s1)))
    Weight_in[i]=w_in
    
    s2=set(tg.successors(i))
    w_out = 1.0/(np.sqrt(1+len(s2)))
    Weight_out[i]=w_out
    
#for imputing with mean
mean_weight_in = np.mean(list(Weight_in.values()))
mean_weight_out = np.mean(list(Weight_out.values()))

100%|██████████| 1780722/1780722 [00:48<00:00, 37065.74it/s]


In [53]:
# Feature 1
def weight_features(u,v):
    one = Weight_in.get(u,mean_weight_in)
    two = Weight_out.get(v,mean_weight_out)
    three = one + two
    four = one * two
    five = (2*one)+two
    six = one+(two*2)
    return one,two,three,four,five,six
xtrain['wt_1'],xtrain['wt_2'],xtrain['wt_3'],xtrain['wt_4'],xtrain['wt_5'],xtrain['wt_6'] = zip(*xtrain.apply(lambda x: weight_features(x[0],x[1]),axis=1))
xtest['wt_1'],xtest['wt_2'],xtest['wt_3'],xtest['wt_4'],xtest['wt_5'],xtest['wt_6'] = zip(*xtest.apply(lambda x: weight_features(x[0],x[1]),axis=1))

In [54]:
#Feature 2
xtrain['pagerank_s'] = xtrain[0].apply(lambda x: pr.get(x,mean_pr))
xtest['pagerank_s'] = xtest[0].apply(lambda x: pr.get(x,mean_pr))

#Feature 3
xtrain['pagerank_d'] = xtrain[1].apply(lambda x: pr.get(x,mean_pr))
xtest['pagerank_d'] = xtest[1].apply(lambda x: pr.get(x,mean_pr))

#Freature 4
xtrain['katz_s'] = xtrain[0].apply(lambda x: katz.get(x,mean_katz))
xtest['katz_s'] = xtest[0].apply(lambda x: katz.get(x,mean_katz))

#Feature 5Adj = nx.adjacency_matrix(train_graph,nodelist=sorted(train_graph.nodes())).asfptype()
xtrain['katz_d'] = xtrain[1].apply(lambda x: katz.get(x,mean_katz))
xtest['katz_d'] = xtest[1].apply(lambda x: katz.get(x,mean_katz))

#Feature 6
xtrain['hubs_s'] = xtrain[0].apply(lambda x: hits[0].get(x,0))
xtest['hubs_s'] = xtest[0].apply(lambda x: hits[0].get(x,0))

#Feature 7
xtrain['hubs_d'] = xtrain[1].apply(lambda x: hits[0].get(x,0))
xtest['hubs_d'] = xtest[1].apply(lambda x: hits[0].get(x,0))

#Feature 9
xtrain['authorities_s'] = xtrain[0].apply(lambda x: hits[1].get(x,0))
xtest['authorities_s'] = xtest[0].apply(lambda x: hits[1].get(x,0))

#Feature 10
xtrain['authorities_d'] = xtrain[1].apply(lambda x: hits[1].get(x,0))
xtest['authorities_d'] = xtest[1].apply(lambda x: hits[1].get(x,0))

## 5.5 Adding new set of features

__we will create these each of these features for both train and test data points__
<ol>
<li>SVD features for both source and destination</li>
</ol>

In [55]:
Adj = nx.adjacency_matrix(tg,nodelist=sorted(tg.nodes())).asfptype()

In [56]:
U, s, V = svds(Adj, k = 6) 
print('Adjacency matrix Shape',Adj.shape)
print('U Shape',U.shape)
print('V Shape',V.shape)
print('s Shape',s.shape)

Adjacency matrix Shape (1780722, 1780722)
U Shape (1780722, 6)
V Shape (6, 1780722)
s Shape (6,)


We have got two feature sets i.e U & V
Since K=6 each node say source will have an 6 dimentional U feature and then same with V. Similarly the destination node will have 6 n 6 dimentional feature.
Therefore we will have new 6*4=24 dimentional feature.

In [57]:
'''
When we make use of svds of scipy we get U,V. But now we no longer have the names of the nodes. So we need to first stored the nodes with it's index so that we can get the index of U/V when we have the value/no. of an particulat node.
'''
def svd(x, S):
    try:
        z = sadj_dict[x]
        return S[z]
    except:
        return [0,0,0,0,0,0]

#for svd features to get feature vector creating a dict node val and inedx in svd vector
sadj_col = sorted(tg.nodes())
sadj_dict = { val:idx for idx,val in enumerate(sadj_col)}

In [60]:
xtrain['svd_u_s_1'],xtrain['svd_u_s_2'],xtrain['svd_u_s_3'],xtrain['svd_u_s_4'],xtrain['svd_u_s_5'],xtrain['svd_u_s_6'] = zip(*xtrain[0].apply(lambda x : svd(x,U)))
xtest['svd_u_s_1'],xtest['svd_u_s_2'],xtest['svd_u_s_3'],xtest['svd_u_s_4'],xtest['svd_u_s_5'],xtest['svd_u_s_6'] = zip(*xtest[0].apply(lambda x : svd(x,U)))

xtrain['svd_u_d_1'],xtrain['svd_u_d_2'],xtrain['svd_u_d_3'],xtrain['svd_u_d_4'],xtrain['svd_u_d_5'],xtrain['svd_u_d_6'] = zip(*xtrain[1].apply(lambda x : svd(x,U)))
xtest['svd_u_d_1'],xtest['svd_u_d_2'],xtest['svd_u_d_3'],xtest['svd_u_d_4'],xtest['svd_u_d_5'],xtest['svd_u_d_6'] = zip(*xtest[1].apply(lambda x : svd(x,U)))

xtrain['svd_v_s_1'],xtrain['svd_v_s_2'],xtrain['svd_v_s_3'],xtrain['svd_v_s_4'],xtrain['svd_v_s_5'],xtrain['svd_v_s_6'] = zip(*xtrain[0].apply(lambda x : svd(x,V.T)))
xtest['svd_v_s_1'],xtest['svd_v_s_2'],xtest['svd_v_s_3'],xtest['svd_v_s_4'],xtest['svd_v_s_5'],xtest['svd_v_s_6'] = zip(*xtest[0].apply(lambda x : svd(x,V.T)))

xtrain['svd_v_d_1'],xtrain['svd_v_d_2'],xtrain['svd_v_d_3'],xtrain['svd_v_d_4'],xtrain['svd_v_d_5'],xtrain['svd_v_d_6'] = zip(*xtrain[1].apply(lambda x : svd(x,V.T)))
xtest['svd_v_d_1'],xtest['svd_v_d_2'],xtest['svd_v_d_3'],xtest['svd_v_d_4'],xtest['svd_v_d_5'],xtest['svd_v_d_6'] = zip(*xtest[1].apply(lambda x : svd(x,V.T)))

In [62]:
xtrain.columns

Index([                0,                 1,               'y',
         'jacc_followee',   'jacc_follower', 'cosine_followee',
       'cosine_follower', 'num_followers_s', 'num_followers_d',
       'num_followees_s', 'num_followees_d', 'inter_followers',
       'inter_followees',      'adar_index',     'follow_back',
              'same_wcc',            'wt_1',            'wt_2',
                  'wt_3',            'wt_4',            'wt_5',
                  'wt_6',      'pagerank_s',      'pagerank_d',
                'katz_s',          'katz_d',          'hubs_s',
                'hubs_d',   'authorities_s',   'authorities_d',
             'svd_u_s_1',       'svd_u_s_2',       'svd_u_s_3',
             'svd_u_s_4',       'svd_u_s_5',       'svd_u_s_6',
             'svd_u_d_1',       'svd_u_d_2',       'svd_u_d_3',
             'svd_u_d_4',       'svd_u_d_5',       'svd_u_d_6',
             'svd_v_s_1',       'svd_v_s_2',       'svd_v_s_3',
             'svd_v_s_4',       'svd_v_s

In [63]:
xtest.columns

Index([                0,                 1,               'y',
         'jacc_followee',   'jacc_follower', 'cosine_followee',
       'cosine_follower', 'num_followers_s', 'num_followers_d',
       'num_followees_s', 'num_followees_d', 'inter_followers',
       'inter_followees',      'adar_index',     'follow_back',
              'same_wcc',            'wt_1',            'wt_2',
                  'wt_3',            'wt_4',            'wt_5',
                  'wt_6',      'pagerank_s',      'pagerank_d',
                'katz_s',          'katz_d',          'hubs_s',
                'hubs_d',   'authorities_s',   'authorities_d',
             'svd_u_s_1',       'svd_u_s_2',       'svd_u_s_3',
             'svd_u_s_4',       'svd_u_s_5',       'svd_u_s_6',
             'svd_u_d_1',       'svd_u_d_2',       'svd_u_d_3',
             'svd_u_d_4',       'svd_u_d_5',       'svd_u_d_6',
             'svd_v_s_1',       'svd_v_s_2',       'svd_v_s_3',
             'svd_v_s_4',       'svd_v_s

In [64]:
xtrain.to_csv('datasets/my/featured_train.csv')
xtest.to_csv('datasets/my/featured_test.csv')

# Extra Features


1. Add another feature called  Preferential Attachment  with followers and followees data of vertex. you can check about Preferential Attachment in below link
http://be.amazd.com/link-prediction/ <br>
2. Add  feature called svd_dot. you can calculate svd_dot as Dot product between sourse node svd and destination node svd features.  you can read about this in below pdf 
https://storage.googleapis.com/kaggle-forum-message-attachments/2594/supervised_link_prediction.pdf<br>